In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

#from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [9]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable first GPU
  tf.config.set_visible_devices(physical_devices[1:], 'GPU')
  logical_devices = tf.config.list_logical_devices('GPU')
  # Logical device was not created for first GPU
  assert len(logical_devices) == len(physical_devices) - 1
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [4]:
df = pd.read_csv('data/single_step_df_ints_2022-04-06.csv')

In [5]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,spice_shaker.already_seen,ginger.already_seen,carrot.already_seen,apricots.already_seen,sugar.already_seen,...,lemon_juice_container.food_k,lemon_juice_container.strong_k,lemon_juice_container.mid_k,coordinates_bowl2.x,coordinates_bowl2.y,coordinates_bowl2.z,bowl2.containment,bowl2.food_k,bowl2.strong_k,bowl2.mid_k
0,<start>,plate,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plate,plate_small,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,plate_small,cup,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cup,glass,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glass,knife,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,pineapple,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,pineapple,cutting_board,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,cutting_board,knife,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,knife,plastic_box,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [7]:
df['target'].unique()
#df['target'].value_counts()

array(['plate', 'plate_small', 'cup', 'glass', 'knife', 'spoon_small',
       'wineglass', 'fork', 'spoon', 'tray', 'fork_small', 'napkin',
       'silverware', 'placemat', 'container', 'table_cloth', 'cereal',
       'cutting_board', 'can', 'bowl', 'bottle', 'tomato_plate', 'grapes',
       'milk', 'yoghurt', 'blueberries', 'bananas', 'apricots',
       'tupperware', 'bread', 'salad', 'oil', 'water', 'beer', 'cheese',
       'plate2', 'food', 'drink', 'cucumber', 'orange', 'garlic_press',
       'garlic', 'teaspoon', 'squeezer', 'lime', 'pepper', 'chilli',
       'kiwi', 'mango', 'peeler', 'cauliflower', 'colander', 'onion',
       'plum', 'potato', 'pineapple', 'plastic_box', 'beans', 'pan',
       'spice_shaker', 'spatula', 'lid', 'broccoli', 'pot', 'jar',
       'avocado', 'fig', 'egg', 'carrots', 'leek', 'herbs', 'ginger',
       'carrot', 'oranges', 'pomegranate', 'chef_knife', 'butter', 'salt',
       'flour', 'wire_whisk', 'ladle', 'mug', 'sugar', 'paper_bag',
       'spice', '

In [8]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [9]:
# shuffle

df_new = df.sample(frac=1)

df_new

,input,target,start_coords.x,start_coords.y,start_coords.z,spice_shaker.already_seen,ginger.already_seen,carrot.already_seen,apricots.already_seen,sugar.already_seen,...,lemon_juice_container.food_k,lemon_juice_container.strong_k,lemon_juice_container.mid_k,coordinates_bowl2.x,coordinates_bowl2.y,coordinates_bowl2.z,bowl2.containment,bowl2.food_k,bowl2.strong_k,bowl2.mid_k
1440,spoon,cereal,0.000000,2.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
647,spoon_small,fork_small,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2028,plate,knife,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1095,bowl,knife,5.500000,4.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1670,knife,plate,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,<start>,mango,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
860,glass,bottle,5.500000,4.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
310,plate,tray,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
479,wineglass,glass,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [10]:
train = df[:int(round(len(df)*0.7,0))]
test_val = df[int(round(len(df)*0.7,0)):]

In [11]:
test_val

,input,target,start_coords.x,start_coords.y,start_coords.z,spice_shaker.already_seen,ginger.already_seen,carrot.already_seen,apricots.already_seen,sugar.already_seen,...,lemon_juice_container.food_k,lemon_juice_container.strong_k,lemon_juice_container.mid_k,coordinates_bowl2.x,coordinates_bowl2.y,coordinates_bowl2.z,bowl2.containment,bowl2.food_k,bowl2.strong_k,bowl2.mid_k
1471,knife,orange,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1472,orange,plate,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1473,<start>,garlic_press,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1474,garlic_press,plate,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1475,plate,garlic,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,pineapple,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2097,pineapple,cutting_board,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2098,cutting_board,knife,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2099,knife,plastic_box,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [12]:
# split test_val into test and val

split_test_val = len(test_val)//2

test = test_val[:split_test_val]
val = test_val[split_test_val:]

In [13]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [14]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [15]:
batch_size = 128

In [16]:
train_ds = create_dataset(train, labels, batch_size=batch_size)
val_ds = create_dataset(val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(test, labels, shuffle=False, batch_size=batch_size)

/home/pwenzl/anaconda3/envs/tf2.7/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:875: UserWarning: unknown class(es) ['2', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y'] will be ignored
  warnings.warn(
/tmp/ipykernel_8375/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}
2022-04-07 11:08:27.231270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 11:08:27.260675: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA no

In [23]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch)

Every feature: ['input', 'start_coords.x', 'start_coords.y', 'start_coords.z', 'spice_shaker.already_seen', 'ginger.already_seen', 'carrot.already_seen', 'apricots.already_seen', 'sugar.already_seen', 'salt.already_seen', 'garlic_press.already_seen', 'pot2.already_seen', 'spatula.already_seen', 'chilli.already_seen', 'potato.already_seen', 'chef_knife.already_seen', 'drink.already_seen', 'spoon_small.already_seen', 'milk.already_seen', 'food.already_seen', 'plate.already_seen', 'fork.already_seen', 'plate2.already_seen', 'silverware.already_seen', 'peeler.already_seen', 'pomegranate.already_seen', 'knife.already_seen', 'wineglass.already_seen', 'bowl2.already_seen', 'mug.already_seen', 'wire_whisk.already_seen', 'can.already_seen', 'bread.already_seen', 'measuring_pitcher.already_seen', 'broccoli.already_seen', 'tray.already_seen', 'fork_small.already_seen', 'tupperware.already_seen', 'spoon.already_seen', 'paper_bag.already_seen', 'pepper.already_seen', 'cereal.already_seen', 'colande

In [20]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [21]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [22]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [23]:
all_inputs, encoded_features = create_input_data(df)

In [31]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

In [28]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(93)(x)

model = tf.keras.Model(all_inputs, output)

In [29]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [33]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[])

Epoch 1/100
12/12 [==============================] - 2s 123ms/step - loss: 0.0000e+00 - accuracy: 0.0088 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
12/12 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.0129 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/100
12/12 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.0075 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/100
12/12 [==============================] - 2s 123ms/step - loss: 0.0000e+00 - accuracy: 0.0129 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/100
12/12 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 0.0109 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/100
12/12 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.0068 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/100
12/12 [==============================] - 2s 121ms/step - l

12/12 [==============================] - 2s 123ms/step - loss: 0.0000e+00 - accuracy: 0.0102 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 54/100
12/12 [==============================] - 2s 121ms/step - loss: 0.0000e+00 - accuracy: 0.0122 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 55/100
12/12 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.0109 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 56/100
12/12 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.0109 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 57/100
12/12 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.0102 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 58/100
12/12 [==============================] - 2s 123ms/step - loss: 0.0000e+00 - accuracy: 0.0102 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 59/100
12/12 [==============================] - 2s 124ms/step - loss: 0

In [22]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 98ms/step - loss: 1.3626 - accuracy: 0.5251
Accuracy 0.5251141786575317


In [28]:
model.save('models/next_obj_classifier_2022-03-23')

2022-03-23 12:41:32.131340: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


In [31]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_2022-03-23')

In [ ]:
# test prediction for one sample

In [32]:
sample = df.loc[0].drop('target').to_dict()

In [33]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [34]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [35]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [36]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(0, len(dataframe)): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [37]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

In [38]:
summed_error = [sum(error) for error in errors[:-1]]

In [39]:
np.median(summed_error)

2.0

In [41]:
#with open('results/nn_spatialinfo_prequential_summed_2022-03-23.txt', 'w') as file:
#    file.write(str(summed_error))